In [1]:
from langgraph.graph import Graph, START, END
from typing import TypedDict, List, Annotated, Literal
class State(TypedDict):
    typeOfQuestion : Literal['Weather', 'Course', 'General']
    weather : str
    CourseResult : str
    result : str
    Query : str

In [2]:
from langchain.chat_models import init_chat_model

model =  init_chat_model(
    "gemini-2.0-flash",
    model_provider= "google_genai"
)


In [ ]:
def getCourseInfo(state: State):
    response = graph.invoke({
        "question" : state["Query"]
    })
    print("COURSE RESULT :  " + response["answer"])
    return {
        "CourseResult" : response["answer"]
    }

In [4]:
def getQuery(state: State):
    userInput = input()
    return {
        "Query" : userInput
    }

In [5]:
def ClassifyQuery(state: State):
    from langchain_core.prompts import PromptTemplate

    prompt_template = PromptTemplate.from_template("Classify if its related to Weather or Course or General: {query}. and provide output in a single word")

    prompt = prompt_template.invoke({
        "query" : state["Query"]
    })
    response = model.invoke(prompt)
    print("CLASSIFYING QUERY : " + response.content)
    return response.content

In [6]:
from pydantic import BaseModel, Field
class ResponseFormatter(BaseModel):
    city: str = Field(description="If the User asks for weather in a particular city, fill this field with the city without any spelling errors, else fill it as None")
    Coordinates: List[float] = Field(description="If the user inputs latitude and longitude of a location, return the latitude and longitude as a list of arrays")

    

In [7]:
def predictWeather(state: State):
    
    

    import time
    import weatherapi
    from weatherapi.rest import ApiException
    from pprint import pprint

    model_OS = model.with_structured_output(ResponseFormatter)
    structured_Output = model_OS.invoke(state["Query"])
    structured_Output

    configuration = weatherapi.Configuration()
    configuration.api_key['key'] = 'e00b8e34d43b4cbea5380937250906'
    api_instance = weatherapi.APIsApi(weatherapi.ApiClient(configuration))
    # q = str(lat) + ',' + str(lng) # str | Pass US Zipcode, UK Postcode, Canada Postalcode, IP address, Latitude/Longitude (decimal degree) or city name. Visit [request parameter section](https://www.weatherapi.com/docs/#intro-request) to learn more.
    dt = '2025-05-09' # date | Date on or after 1st Jan, 2015 in yyyy-MM-dd format
    
    if structured_Output.city== 'None':
        lat, lng = structured_Output.Coordinates
        q = str(lat) + "," + str(lng) 
        print("GIVEN COORDINATES : " + q)
    else:
        q = structured_Output.city
        print("GIVEN CITY : " + q)

    try:
        # Astronomy API
        api_response = api_instance.forecast_weather(q, dt)
        pprint(api_response['current']['condition']['text'])
        return {
            "weather" : api_response['current']['condition']['text']
        }
    except ApiException as e:
           print("Exception when calling APIsApi->astronomy: %s\n" % e)

In [8]:

from langchain_core.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template("Classify if its related to Weather or Course: {query}. and provide output in a single word")
prompt = prompt_template.invoke({
    "query" : "no of credits for 22CS610" 
})
response = model.invoke(prompt)
response.content

'Course'

In [9]:
def GeneralQuery(state: State):
    response = model.invoke(state["Query"])
    print(response.content)
    return {
        "result": response.content
    } 

In [ ]:
from langgraph.graph import START, END, StateGraph

graphBuilder =  StateGraph(State)
graphBuilder.add_node("getQuery", getQuery)
# graphBuilder.add_node("ClassifyQuery", ClassifyQuery)
graphBuilder.add_node("Weather", predictWeather)
graphBuilder.add_node("Course", getCourseInfo)
graphBuilder.add_node("General", GeneralQuery)

graphBuilder.add_edge(START, "getQuery")
graphBuilder.add_conditional_edges("getQuery", ClassifyQuery, {
    "Weather": "Weather",
    "Course": "Course", 
    "General": "General"
})
graphBuilder.add_edge("Weather", "getQuery")
graphBuilder.add_edge("Course", "getQuery")
graphBuilder.add_edge("General", "getQuery")

graph = graphBuilder.compile()
graph.invoke(input= {
    "typeOfQuestion" : "Weather",
    "weather" : "",
    "CourseResult" : "",
    "Query": "",
    "result": ""
}) 



CLASSIFYING QUERY : General
Okay, Sai! It's nice to meet you. How can I help you today?
CLASSIFYING QUERY : General
As a large language model, I don't have access to personal information. Therefore, I don't know your name. You haven't told me!
CLASSIFYING QUERY : General
Okay! Is there anything specific you'd like me to do with "fghjk"? 

For example, are you:

*   **Testing if I can read it?** (I can!)
*   **Wanting me to spell it out?** (F-G-H-J-K)
*   **Looking for words that contain these letters?**
*   **Trying to start a conversation?**

Let me know what you're thinking!
CLASSIFYING QUERY : General
Okay, let's convert 2 billion US dollars to Indian Rupees and express it in words.

**1. Conversion Rate:**

As of November 2nd, 2024, the approximate exchange rate is:

1 USD = 83.32 INR (This rate fluctuates, so it's always best to check a reliable source like Google Finance, a bank, or a currency converter for the most up-to-date rate.)

**2. Calculation:**

2,000,000,000 USD * 83.3

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))